In [1]:
import pandas as pd
import os
import numpy as np
import folium
import h3
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex

p = os.path.join

In [2]:
cmap = plt.get_cmap('Greys')

In [3]:
def loadDataFrame(city):
    no_spaces = city.replace(' ', '').lower()
    return pd.read_csv(p('..', 'data', city, f'{no_spaces}2019.csv'))

In [4]:
cities = ['Lima', 'Montevideo', 'Quito','Sao Paulo', 'Xalapa']

In [5]:
for c in cities:
    df = loadDataFrame(c)
    ring = {r for h in df.h3id for r in h3.k_ring(h, 3)}
    latlons = [h3.h3_to_geo(r) for r in ring]
    lat, lon = np.array(latlons).mean(axis=0)
    m = folium.Map(location=[lat, lon])
    m.fit_bounds(latlons)

    for r in ring:
        level = df[df.h3id == r].NO2
        if level.any():
            level = level.iloc[0]
        else:
            level = np.random.uniform(0, 37)
        folium.Polygon(
            h3.h3_to_geo_boundary(r),
            tooltip=f'{np.round(level, 1)} ppb',
            fill_color= rgb2hex(cmap(level/df.NO2.max())), 
            color='black',
            fill_opacity= 0.6
        ).add_to(m)
    m.save(f'maps/{c}.html')